In [22]:
# -*- coding:utf-8 -*-
import webbrowser
import requests
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

# gen url to jisho
def webInfo(kan):
    webstem = 'https://jisho.org/search/'
    suffix = ""
    k = str((kan).encode("utf-8"))
    kanji = k[2:-1]
    chars = "\\x"
    kanji = kanji.replace(chars, '%')
    webstring = webstem + kanji + suffix
    return webstring


# Load webpage content
def jlptGrade(webstring):
    r = requests.get(webstring)
    soup = bs(r.content, features="html.parser")
    try:
        n = soup.find("span", string=re.compile('(JLPT|jlpt)')).text
        grade = n.split(' ')[1]
    except:
        grade = 'NA'
    return grade

In [23]:
# get csv to df for preparation
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/word_dictionary.csv'
df = pd.read_csv(path)
kanjis = df['kanji']




In [24]:
# scrape grades
# get prev scraped grades
def get_grades(df,kanjis):
    prevs= pd.read_csv('kanji_data_cleansed.csv')
    grades = list(prevs.grade)
    maxer = prevs['word index'].max()

    for i,kanji in enumerate(kanjis):
        if i > maxer:
            if i % 10 ==0 or i == maxer + 1:
                print(i, kanji)
            webstring = webInfo(kanji)
            grade = jlptGrade(webstring)
            grades.append(grade)
    print(grades)
    df['grade'] = grades
    print(df.grade.value_counts())
    return df

613 報告
['N5', 'N4', 'N3', 'N5', 'N5', nan, 'N4', 'N4', nan, 'N3', 'N5', 'N3', 'N3', 'N3', 'N5', 'N5', nan, nan, 'N4', 'N3', 'N4', 'N5', 'N5', 'N4', 'N5', 'N3', 'N4', nan, 'N5', 'N5', 'N3', 'N3', 'N4', 'N4', 'N5', 'N5', 'N2', nan, 'N5', 'N5', 'N2', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N4', 'N5', 'N3', 'N4', 'N5', 'N4', nan, 'N3', 'N5', 'N5', 'N5', 'N3', 'N3', 'N5', 'N3', nan, 'N4', 'N2', 'N4', 'N3', nan, 'N5', 'N3', 'N4', nan, 'N4', 'N1', 'N5', 'N4', 'N5', 'N5', 'N2', 'N5', 'N5', 'N3', 'N5', 'N4', 'N3', 'N4', 'N2', 'N3', 'N2', 'N5', 'N1', 'N4', 'N3', nan, 'N4', 'N2', nan, 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N3', 'N4', 'N4', 'N5', 'N5', 'N5', 'N5', 'N5', 'N3', 'N4', 'N5', 'N5', nan, 'N5', nan, 'N4', 'N4', 'N5', 'N5', 'N5', nan, 'N5', nan, nan, 'N5', 'N5', 'N5', 'N3', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', nan, 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N5', 'N

N5    245
N4    138
N3     93
N2     32
N1     26
NA      3
Name: grade, dtype: int64

In [26]:
# get streak info
def get_streak_info(df):
    path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/streak.csv'
    df_str = pd.read_csv(path)
    df['streak'] = df_str.streak
    df['level'] = df.streak.apply(lambda x: 'expert' if x > 7 else 'advanced' if x > 2 else 'beginner')
    return df

In [27]:
# get results info
def get_results_info(df):
    path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/results2.csv'
    df_res = pd.read_csv(path)
    fails = df_res.groupby('index_word')['result'].value_counts().unstack().fillna(0)[0]
    successes = df_res.groupby('index_word')['result'].value_counts().unstack().fillna(0)[1]
    df['total_attempts'] = (fails + successes)
    df['total_successes'] = successes
    df['total_fails'] = fails
    df['success_ratio'] = df['total_successes']/df['total_attempts']
    df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,success_ratio
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,1.000000
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,1.000000
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,1.000000
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,51.0,41.0,10.0,0.803922
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,0.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,623,absence;not at home,るす,留守,03/03/2021,3,4,1,N4,0,beginner,1.0,0.0,1.0,0.000000
624,624,far (away),とおい,遠い,03/03/2021,3,4,1,N5,1,beginner,1.0,1.0,0.0,1.000000
625,625,safe,あんぜん,安全,03/03/2021,2,5,1,N4,0,beginner,2.0,1.0,1.0,0.500000
626,626,precious;valuable,たいせつ,大切,03/03/2021,1,4,1,N5,1,beginner,1.0,1.0,0.0,1.000000


In [28]:
# age column
from datetime import datetime as dt

today_ = pd.to_datetime('today').normalize()

df['age'] = df.date_added.apply(lambda x: today_ - pd.to_datetime(dt.strptime(x,"%d/%m/%Y"))).dt.days
df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,success_ratio,age
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,1.000000,299
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,1.000000,299
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,1.000000,299
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,51.0,41.0,10.0,0.803922,299
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,0.833333,299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,623,absence;not at home,るす,留守,03/03/2021,3,4,1,N4,0,beginner,1.0,0.0,1.0,0.000000,1
624,624,far (away),とおい,遠い,03/03/2021,3,4,1,N5,1,beginner,1.0,1.0,0.0,1.000000,1
625,625,safe,あんぜん,安全,03/03/2021,2,5,1,N4,0,beginner,2.0,1.0,1.0,0.500000,1
626,626,precious;valuable,たいせつ,大切,03/03/2021,1,4,1,N5,1,beginner,1.0,1.0,0.0,1.000000,1


In [29]:
df.to_csv('kanji_data_cleansed.csv', index=False)

In [30]:
df

,word index,english,hiragana,kanji,date_added,rank,sample,is_active,grade,streak,level,total_attempts,total_successes,total_fails,success_ratio,age
0,0,house,いえ,家,09/05/2020,1,1,0,N5,10,expert,19.0,19.0,0.0,1.000000,299
1,1,(someone's) house,おたく,お宅,09/05/2020,1,1,0,N4,10,expert,12.0,12.0,0.0,1.000000,299
2,2,nervous,ふあん,不安,09/05/2020,2,1,0,N3,10,expert,17.0,17.0,0.0,1.000000,299
3,3,class,じゅぎょう,授業,09/05/2020,3,1,1,N5,8,expert,51.0,41.0,10.0,0.803922,299
4,4,to lend,かす,貸す,09/05/2020,2,1,0,N5,10,expert,42.0,35.0,7.0,0.833333,299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,623,absence;not at home,るす,留守,03/03/2021,3,4,1,N4,0,beginner,1.0,0.0,1.0,0.000000,1
624,624,far (away),とおい,遠い,03/03/2021,3,4,1,N5,1,beginner,1.0,1.0,0.0,1.000000,1
625,625,safe,あんぜん,安全,03/03/2021,2,5,1,N4,0,beginner,2.0,1.0,1.0,0.500000,1
626,626,precious;valuable,たいせつ,大切,03/03/2021,1,4,1,N5,1,beginner,1.0,1.0,0.0,1.000000,1


In [16]:
path = '/Users/mattcollins/Projects/MyProject/KanjiProgram/csvs/fact_streak.csv'
df2 = pd.read_csv(path)
df2

,index_word,streak,date,sample
0,1,8,04/08/2020,1
1,5,8,04/08/2020,1
2,16,8,04/08/2020,1
3,20,7,04/08/2020,1
4,39,8,04/08/2020,1
...,...,...,...,...
5617,608,1,19/02/2021,5
5618,609,1,19/02/2021,4
5619,610,0,19/02/2021,5
5620,611,1,19/02/2021,4
